Depoly MyToken contract

In [9]:
from web3 import Web3
from eth_account import Account
import json
import os
from dotenv import load_dotenv
load_dotenv()

# Connect to Hyperliquid testnet
w3 = Web3(Web3.HTTPProvider('https://api.hyperliquid-testnet.xyz/evm'))

# Your private key (keep this secret and safe!)
private_key = os.getenv('DEPLOYER_PRIVATE_KEY')
account = Account.from_key(private_key)

# Load compiled MyToken contract data
with open('./bin/MyToken.abi', 'r') as f:
    abi = json.loads(f.read())
    
with open('./bin/MyToken.bin', 'r') as f:
    bytecode = '0x' + f.read()

# Create contract instance
MyToken = w3.eth.contract(abi=abi, bytecode=bytecode)

# Set initial supply (1 million tokens)
initial_supply = 1000000  # This will be multiplied by 10^18 in the contract

# Build deployment transaction
transaction = MyToken.constructor(initial_supply).build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 2000000,  # Increased gas limit for ERC20 deployment
    'gasPrice': w3.eth.gas_price
})

# Sign transaction
signed = w3.eth.account.sign_transaction(transaction, private_key)

# Send transaction
tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)

# Wait for transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Token contract deployed at: {tx_receipt.contractAddress}")

# Create contract instance at deployed address for interaction
token = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

# Verify deployment by checking total supply
total_supply = token.functions.totalSupply().call()
print(f"Total supply: {total_supply / 10**18} MTK")  # Divide by 10^18 to get actual token amount

Token contract deployed at: 0xDA07B9eFd8BDF490301C558f7D9707FBed8a73ed
Total supply: 1000000.0 MTK


Test the deployed contract

In [15]:
# Create contract instance at deployed address
contract_address = '0xDA07B9eFd8BDF490301C558f7D9707FBed8a73ed'  # The address from your deployment
token = w3.eth.contract(address=contract_address, abi=abi)

# Read total supply
total_supply = token.functions.totalSupply().call()
print(f"Total supply: {total_supply / 10**18} MTK")

# Get token balance of deployer account
balance = token.functions.balanceOf(account.address).call()
print(f"Deployer balance: {balance / 10**18} MTK")

# Optional: Transfer some tokens to another address
recipient = '0xF920F86F7B78e7f2081e90Aaf7eb2F1D50f49125'  # Replace with actual recipient address
amount_to_transfer = Web3.to_wei(100, 'ether')  # Transfer 100 tokens

transfer_txn = token.functions.transfer(recipient, amount_to_transfer).build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 100000,
    'gasPrice': w3.eth.gas_price
})

# Sign and send transaction
signed_txn = w3.eth.account.sign_transaction(transfer_txn, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Wait for transaction to be mined
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Transfer transaction successful! Hash: {tx_hash.hex()}")

# Check new balances
new_sender_balance = token.functions.balanceOf(account.address).call()
new_recipient_balance = token.functions.balanceOf(recipient).call()
print(f"New deployer balance: {new_sender_balance / 10**18} MTK")
print(f"Recipient balance: {new_recipient_balance / 10**18} MTK")

Total supply: 1000000.0 MTK
Deployer balance: 999900.0 MTK
Transfer transaction successful! Hash: 55baa8f8d9d9bce78c9f649910437a226028a6ede076d00d44a11a1059a3016c
New deployer balance: 999900.0 MTK
Recipient balance: 999900.0 MTK
